This notebook computes outputs of a few different models in a hacky adhoc way. It loads a model (specified in the imports) and its weights from a specific pickle file path.
The data path determines which input npy files are used.
I ran this to quickly generate (and evaluate) outputs of the models trained for different melody definitions.

In [ ]:
cd ../deepsalience/

In [ ]:
# from multif0_exper11_batchin import model_def
from multif0_exper11_batchin_mel2 import model_def
import core
import medleydb as mdb
import evaluate
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# pkl_path = "../experiment_output/multif0_exper11_batchin/multif0_exper11_batchin.pkl"
# pkl_path = "../experiment_output/multif0_exper11_batchin_mel3/multif0_exper11_batchin_mel3.pkl"
# pkl_path = "../experiment_output/multif0_exper11_batchin_mf0incomplete/multif0_exper11_batchin_mf0incomplete.pkl"
pkl_path = "../experiment_output/multif0_exper11_batchin_mel2/multif0_exper11_batchin_mel2.pkl"


In [ ]:
model = model_def()
model.load_weights(pkl_path)

In [ ]:
model.summary()

In [ ]:
# mf0_incomplete_path = core.data_path_multif0_incomplete()
# mf0_mel3_path = core.data_path_melody3()
mf0_mel2_path = core.data_path_melody2()
output_path = "../experiment_output_submission/Model_11b_mel2_outputs/"
mtracks = list(mdb.load_all_multitracks(dataset_version=['V1']))

In [ ]:
for mtrack in mtracks:
    if mtrack.melody1_annotation is not None:
        print(mtrack.track_id)
        # get input numpy file
        input_npy = glob.glob(
            os.path.join(mf0_mel2_path, 'inputs', '{}*.npy'.format(mtrack.track_id))
        )[0]
        
        # get predicted output
        print("    Getting prediction...")
        predicted_output, input_hcqt = evaluate.get_single_test_prediction(model npy_file=input_npy)
        
        # save output numpy
        print("    Saving prediction...")
        np.save(
            os.path.join(output_path, '{}_prediction.npy'.format(mtrack.track_id)),
            predicted_output.astype(np.float32)
        )
        
        print("    Plotting output...")
        plt.figure(figsize=(15, 15))

        plt.subplot(2, 1, 1)
        plt.imshow(input_hcqt[0, :, :, 1], origin='lower')
        plt.axis('auto')
        plt.colorbar()

        plt.subplot(2, 1, 2)
        plt.imshow(predicted_output, origin='lower')
        plt.axis('auto')
        plt.colorbar()

        plot_save_path = os.path.join(output_path, '{}_plot.pdf'.format(mtrack.track_id))
        plt.savefig(plot_save_path, format='pdf')
        plt.close()
    else:
        print("    > {} does not have melody".format(mtrack.track_id))

In [ ]:
ls /scratch/rmb456/multif0_ismir2017/training_data_with_blur/multif0_incomplete/inputs/